# 构造微调训练数据集

借助 ChatGPT 和 GPT API 我们可以实现自动化批量构造训练数据集。

下面我们以中国古典哲学数据集为例，展示了自动构造训练集的主要流程：

- 使用 LangChain 构造训练数据样例
    - 基于 ChatGPT 设计 `System Role` 提示词
    - 使用 `OpenAI GPT-3.5-Turbo-1106` 生成基础数据
    - 解析 OpenAI GPT 生成的训练数据
    - 持久化存储`dataset.csv`训练数据集文件
    - 使用 ChatGPT 实现训练数据多样化
- 自动化批量生成训练数据集
    - 整理收集原始数据`raw_data.txt`
    - 自动解析原始数据样例 `raw_data_content[]`
    - 设计 `gen_data` 训练数据生成器函数
    - 设计训练数据生成流水线

最佳实践参考：

- 使用 GPT-3.5 生成基础数据：https://platform.openai.com/playground/p/2c7XNPgo6Y2iDxILiWfD3iPu?model=gpt-3.5-turbo-1106&mode=chat
- 使用 ChatGPT 生成数据处理代码和相关文本整理：https://chat.openai.com/share/cdfd2d1d-a75e-4cee-be49-539c010ca1b1
- GPT API 价格: https://openai.com/pricing

## 使用 OpenAI SDk 构造训练数据

In [19]:
from openai import OpenAI
import os
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'),base_url=os.getenv('OPENAI_BASE_URL'))
# print(os.getenv('OPENAI_API_KEY'))

In [20]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {
      "role": "system",
      "content": "你是中国古典哲学大师，尤其擅长周易的哲学解读。\n\n接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。\n\n示例输入：\n\n师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。\n师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。\n\n期待结果：\n\ncontent:\"师卦\"\nsummary:\"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。\n\n师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。\n\n师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。\""
    },
    {
      "role": "user",
      "content": "蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。\n蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。\n《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。\n《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。"
    },
    {
      "role": "assistant",
      "content": "content:\"蒙卦\"\nsummary:\"在周易中，蒙卦象征着教育启蒙和智慧的初期阶段，由两个异卦相叠组成：下卦坎（水）和上卦艮（山）。坎象征水，艮象征山，代表了山下有水源，水流出山。这预示着在事物发展的起初阶段，人们会陷入蒙昧状态，因此教育成为当务之急。只有培养纯正无邪的品质，才能走上通达之路，如同泉水汇成江河。\n\n蒙卦的核心哲学是：事物发展的初期阶段必然处于蒙昧状态，但如果能把握时机，行动切合时宜，就能启蒙通达。蒙卦紧随始生卦《屯》之后，《序卦》中解释道：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”这说明物在幼稚阶段都会经历蒙昧状态，找它蒙即是稚嫩。\n\n《象》中解释说：山下出泉，蒙；君子以果行育德，意味着像泉水从山下流出一样，启蒙教育有助于培育美德。\""
    },
    {
      "role": "user",
      "content": "需卦，本卦是异卦相叠，上卦为坎，下卦为乾。乾卦象征天空辽阔，坎卦象征乌云密布。雨水之于传统农业社会而言，是最大的需求，因此用这个意象来表现。古人引申为雨将下，万物皆待，所以叫需。\n需，古体字结构是“雨上天下”，从雨而声。如苍天下雨，滋养万物及人。从人的角度看，天下雨，则不易出行耕作，因此在家喝酒吃肉、静待天时。所以，这一卦的基本象征就是等待、不进，还有饮食之义。\n \n需卦位于蒙卦之后，《序卦》之中这样解释道：“物稚不可不养也，故受之以需。需者，饮食之道也。”有所需要，也是有所等待。\n《象》曰：云上于天，需;君子以饮食宴乐。\n《象》中这段话的意思是说：需卦的卦象是乾(天)下坎(水)上，为水在天上之表象。水汽聚集天上成为云层，密云満天，但还没有下雨，需要等待;君子在这个时候需要吃喝，饮酒作乐，即在等待的时候积蓄力量。\n需卦象征等待，启示人们守正待机的道理，属于中上卦。《象》这样评断此卦：明珠土埋日久深，无光无亮到如今，忽然大风吹土去，自然显露有重新。\n"
    },
    {
      "role": "assistant",
      "content": "content:\"需卦\"\nsummary:\"在周易中，需卦是一个富有深意的卦象，由两个异卦相叠组成：下卦乾（天）和上卦坎（水）。乾象征广阔的天空，而坎象征乌云密布，暗示着迫切需要雨水的意象。古人将雨水视为万物生长的必需，因此命名为“需”。需卦的基本象征是等待和饮食之道。\n\n需卦位于蒙卦之后，《序卦》中解释为“物稚不可不养也，故受之以需。需者，饮食之道也。”说明物在幼稚状态需要得到滋养，因此受到需卦的启示。《象》中描述：云上于天，需；君子以饮食宴乐。意味着天空乌云密布，需要等待雨水，而君子则在等待的过程中积蓄力量，享受饮食和欢乐。\n\n需卦的核心哲学是：等待，启示着守正待机的道理。它属于中上卦，象征着埋藏已久的明珠，经过大风吹去尘埃后，重新显露光芒。\""
    }
  ],
  temperature=1,
  max_tokens=4095,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [21]:
response

ChatCompletion(id='chatcmpl-9oQ2HITqx3tJZC76SUcIJQnIKUquq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='以上是周易中三个卦象的内容整理和润色，格式符合要求，可以用于大模型训练的输入数据。如果还有其他需要处理的内容，或者对结果有进一步的要求，请继续告诉我。', role='assistant', function_call=None, tool_calls=None))], created=1721803509, model='gpt-3.5-turbo-1106', object='chat.completion', service_tier=None, system_fingerprint='fp_b81a85d4e3', usage=CompletionUsage(completion_tokens=69, prompt_tokens=2472, total_tokens=2541))

## 使用 LangChain 构造训练数据

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

chat = ChatOpenAI(model="gpt-3.5-turbo-1106",
                  temperature=1,
                  max_tokens=4095,
                  top_p=1,
                  frequency_penalty=0,
                  presence_penalty=0)

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


In [23]:
system_content = """
你是中国古典哲学大师，尤其擅长周易的哲学解读。

接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

示例输入：

师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

期待结果：

content:"师卦"
summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
"""


In [24]:
# 原始数据
raw_content = "蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。\n蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。\n《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。\n《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。"

In [25]:
messages = [
    SystemMessage(
        content=system_content
    ),
    HumanMessage(
        content=raw_content
    ),
]

In [26]:
ai_message = chat(messages)

### 解析 OpenAI GPT 生成的训练数据

In [27]:
ai_message.content

'content:"蒙卦"\nsummary:"在周易中，蒙卦代表了教育启蒙的智慧。这一卦象由两个异卦相叠组成：下卦为坎（水），上卦为艮（山）。艮卦象征着山，意味着止；坎卦象征着水，意味着险。这个卦形象征着山下有险，却仍然不停止前进，暗示着蒙昧状态，因此被称为蒙卦。然而，通过把握时机，行动与时俱进，蒙卦具有启蒙和通达的特质。\n\n《蒙》卦是《屯》卦这个始生卦之后的第二卦，《序卦》中解释说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”这意味着在物事发展的初期阶段必然是蒙昧的状态，蒙卦象征着幼稚的阶段，即童蒙阶段。\n\n《象》中解释蒙卦的含义为山下出泉，蒙；君子以果行育德。意味着山下泉水渐次涌出，象征着蒙昧逐渐启蒙的过程；君子应以成果行为来培育人的品德。"'

In [28]:
text = ai_message.content

# 分割字符串来找到content和summary的位置
content_start = text.find('content:"') + len('content:"')
content_end = text.find('"\nsummary:')
summary_start = text.find('summary:"') + len('summary:"')
summary_end = text.rfind('"')

# 提取并存储content和summary
content = text[content_start:content_end].strip()
summary = text[summary_start:summary_end].strip()

print("Content:", content)
print("Summary:", summary)


Content: 蒙卦
Summary: 在周易中，蒙卦代表了教育启蒙的智慧。这一卦象由两个异卦相叠组成：下卦为坎（水），上卦为艮（山）。艮卦象征着山，意味着止；坎卦象征着水，意味着险。这个卦形象征着山下有险，却仍然不停止前进，暗示着蒙昧状态，因此被称为蒙卦。然而，通过把握时机，行动与时俱进，蒙卦具有启蒙和通达的特质。

《蒙》卦是《屯》卦这个始生卦之后的第二卦，《序卦》中解释说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”这意味着在物事发展的初期阶段必然是蒙昧的状态，蒙卦象征着幼稚的阶段，即童蒙阶段。

《象》中解释蒙卦的含义为山下出泉，蒙；君子以果行育德。意味着山下泉水渐次涌出，象征着蒙昧逐渐启蒙的过程；君子应以成果行为来培育人的品德。


### 持久化存储训练数据集文件

In [30]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

# 新建CSV文件并写入数据
with open('test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 写入标题行
    writer.writerow(['content', 'summary'])
    # 写入数据行
    writer.writerow([content, summary])


In [31]:
content

'蒙卦'

In [32]:
summary

'在周易中，蒙卦代表了教育启蒙的智慧。这一卦象由两个异卦相叠组成：下卦为坎（水），上卦为艮（山）。艮卦象征着山，意味着止；坎卦象征着水，意味着险。这个卦形象征着山下有险，却仍然不停止前进，暗示着蒙昧状态，因此被称为蒙卦。然而，通过把握时机，行动与时俱进，蒙卦具有启蒙和通达的特质。\n\n《蒙》卦是《屯》卦这个始生卦之后的第二卦，《序卦》中解释说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”这意味着在物事发展的初期阶段必然是蒙昧的状态，蒙卦象征着幼稚的阶段，即童蒙阶段。\n\n《象》中解释蒙卦的含义为山下出泉，蒙；君子以果行育德。意味着山下泉水渐次涌出，象征着蒙昧逐渐启蒙的过程；君子应以成果行为来培育人的品德。'

### 数据增强：构造多样化的提问方式

In [33]:
def generate_question_summary_pairs(content, summary):
    """
    生成20对提问和总结的配对。

    :param content: 内容（例如：“蒙卦”）。
    :param summary: 内容的总结。
    :return: 包含20对提问和总结的列表。
    """
    # 20种提问模板
    question_templates = [
        "{}代表什么？",
        "周易中的{}含义是什么？",
        "请解释一下{}。",
        "{}在周易中是什么象征？",
        "周易{}的深层含义是什么？",
        "{}和教育启蒙有什么联系？",
        "周易的{}讲述了什么？",
        "{}是怎样的一个卦象？",
        "{}在周易中怎样表达教育的概念？",
        "{}的基本意义是什么？",
        "周易中{}的解释是什么？",
        "{}在周易中代表了哪些方面？",
        "{}涉及哪些哲学思想？",
        "周易中{}的象征意义是什么？",
        "{}的主要讲述内容是什么？",
        "周易{}的核心思想是什么？",
        "{}和启蒙教育之间有何联系？",
        "在周易中，{}象征着什么？",
        "请描述{}的含义。",
        "{}在周易哲学中扮演什么角色？"
    ]

    # 使用content填充提问模板
    questions = [template.format(content) for template in question_templates]

    # 创建提问和总结的配对
    question_summary_pairs = [(question, summary) for question in questions]

    return question_summary_pairs

In [34]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
pairs = generate_question_summary_pairs(content, summary)

# 将结果写入CSV文件
with open('test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['content', 'summary'])
    for pair in pairs:
        writer.writerow(pair)


## 自动化批量生成训练数据流水线

原始数据来源：https://www.zhouyi.cc/zhouyi/yijing64/4103.html

In [35]:
# 初始化一个空列表用于存储原始内容数据
raw_content_data = []

# 读取文件并分割数据样例
with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    # 使用连续的换行符('\n\n')作为分隔符来分割文本
    data_samples = content.split('\n\n')

    # 遍历分割后的数据样例并添加到列表中
    for sample in data_samples:
        # 移除每个样例中的额外空白字符（如果有的话）
        cleaned_sample = sample.strip()
        # 仅添加非空样例
        if cleaned_sample:
            raw_content_data.append(cleaned_sample)

In [36]:
# 输出结果以验证
for i, sample in enumerate(raw_content_data[:5]):  # 打印前5个样例以检查
    print(f"样例 {i+1}:")
    print(sample)
    print("------")


样例 1:
蒙卦原文
蒙。亨。匪我求童蒙，童蒙求我。初筮告，再三渎，渎则不告。利贞。
象曰：山下出泉，蒙。君子以果行育德。
白话文解释
蒙卦：通泰。不是我有求于幼稚愚昧的人，而是幼稚愚昧的人有求于我。第一次占筮，神灵告诉了他。轻慢不敬的再三占筮，轻慢不敬的占筮，神灵就不会告诉他。但还是吉利的卜问。
《象辞》说：上卦为艮，象征山；下卦为坎，象征泉。山下有泉，泉水喷涌而出，这是蒙卦的卦象。君子观此卦象，取法于一往无前的山泉，从而以果敢坚毅的行动来培养自身的品德。
《断易天机》解
蒙卦艮上坎下，为离宫四世卦。蒙即蒙昧，主回还往复，疑惑不前，多忧愁过失，乃是凶卦。
北宋易学家邵雍解
智慧未开，蒙昧闭塞；犹豫不决，缺乏果断。
得此卦者，智慧犹如童蒙，不辨是非，迷失方向；若能顺贤师良友之教，启其聪明则亨通。
台湾国学大儒傅佩荣解
时运：蓄积德行，出而用世。
财运：矿山生意，果决则吉。
家宅：君子居吉；婚姻之始。
身体：驱去邪热，可保平安。
传统解卦
这个卦是异卦（下坎上艮）相叠，艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜，因此，具有启蒙和通达的卦象。
大象：蒙者，昏而无所见也，故宜「启蒙」。
运势：初时迷惑不知方向，须忍耐待机而动，凡事多听取别人意见，则运可通。
事业：事业开始，混乱无序，危机四伏，以勇敢坚毅的行动可以扭转局面。然而必须接受严格教育，培养这种奋发图强的精神。务必脚踏实地，最忌好高骛远，否则会陷入孤立无援的境地。
经商：务必小心谨慎，不得急功近利，尤其应树立高尚的商业道德，以良好的信誉提高竞争力而取胜。
求名：必须接受良好的基础教育，陶冶情操。且动机纯正，可以达到目的。
婚恋：注意考察对方品德，不可以金钱为诱铒。夫妻需相互宽容、理解。
决策：有时会陷入迷惘困顿的境地，加上胆小、不果断，往往误事。如能接受长辈的教诲，甚至严酷的考验，抛弃疑惧的心理，等待适当时机，必然一帆风顺。
------
样例 2:
屯卦原文
屯。元，亨，利，贞。勿用，有攸往，利建侯。
象曰：云，雷，屯；君子以经纶。
白话文解释
屯卦。大吉大利，吉利的占卜。不利于出门。有利于建国封侯。
《象辞》说：屯的上卦为坎，坎为云，下卦为震，震为雷。云行于上，雷动于下，是屯卦的卦象。君子观此卦象，取法于云雷，用云的恩泽，雷的威严来治理国

### 将以上的所有模块，整合到一起，自动化生成数据

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

# 初始化LangChain的GPT-3.5调用
chat = ChatOpenAI(model="gpt-3.5-turbo-1106",
                  temperature=1,
                  max_tokens=4095,
                  top_p=1,
                  frequency_penalty=0,
                  presence_penalty=0)

def gen_data(raw_content):
    """
    使用LangChain GPT-3.5调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-3.5模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


In [38]:
# 示例调用（使用 raw_data.txt 中解析的数据样例）
generated_content = gen_data(raw_content_data[0])
print(generated_content)

content:"蒙卦"
summary:"在周易中，蒙卦由艮卦（山）上卦坎卦（水）组成，代表着通泰的卦象。蒙卦的含义是不是我有求于幼稚愚昧的人，而是幼稚愚昧的人有求于我。初次筮卜告知神灵的意志，但若轻慢不敬地再三渎神，神灵将不再告知。尽管如此，仍是吉利的卜问。

蒙卦在《象辞》中以“山下出泉”来象征，呈现了山下有泉水喷涌的景象。君子观此卦象，以果敢坚毅的行动来培养自身的品德。

北宋易学家邵雍解释蒙卦为蒙昧闭塞，主回还往复，疑惑不前，多忧愁过失，乃是凶卦。

智慧未开，犹豫不决，缺乏果断。得此卦者，智慧犹如童蒙，须通过贤师良友之教，启其聪明则亨通。

蒙卦预示着时运蓄积德行，出而用世；财运则需要果断才能吉利；家宅吉祥，婚姻之始有利，身体驱去邪热可保平安。

传统解卦中提到，蒙卦形成山下有险的卦象，仍不停止前进，显示出启蒙和通达的特质。运势初时迷惑不知方向，需要忍耐待机而动，事业开始混乱无序，危机四伏，需要勇敢坚毅的行动来扭转局面。然而必须接受严格教育，培养这种奋发图强的精神。务必脚踏实地，最忌好高骛远，否则会陷入孤立无援的境地。经商务必小心谨慎，不得急功近利，尤其需树立高尚的商业道德，以良好的信誉提高竞争力。在决策时，有时会陷入迷惘困顿，加上胆小、不果断，往往误事。如能接受长辈的教诲，甚至严酷的考验，抛弃疑惧的心理，等待适当时机，必然一帆风顺。"



In [39]:
def dataset_parser(ai_message_content):
    """
    解析由gen_data函数生成的ai_message.content，提取content和summary。

    :param ai_message_content: gen_data函数返回的文本。
    :return: 提取的content和summary。
    """
    # 分割字符串来找到content和summary的位置
    content_start = ai_message_content.find('content:"') + len('content:"')
    content_end = ai_message_content.find('"\nsummary:')
    summary_start = ai_message_content.find('summary:"') + len('summary:"')
    summary_end = ai_message_content.rfind('"')

    # 提取并存储content和summary
    content = ai_message_content[content_start:content_end].strip()
    summary = ai_message_content[summary_start:summary_end].strip()

    return content, summary


In [40]:
# 示例调用（使用假设的gen_data函数返回的文本）
content, summary = dataset_parser(generated_content)
print("Content:", content)
print("Summary:", summary)

Content: 蒙卦
Summary: 在周易中，蒙卦由艮卦（山）上卦坎卦（水）组成，代表着通泰的卦象。蒙卦的含义是不是我有求于幼稚愚昧的人，而是幼稚愚昧的人有求于我。初次筮卜告知神灵的意志，但若轻慢不敬地再三渎神，神灵将不再告知。尽管如此，仍是吉利的卜问。

蒙卦在《象辞》中以“山下出泉”来象征，呈现了山下有泉水喷涌的景象。君子观此卦象，以果敢坚毅的行动来培养自身的品德。

北宋易学家邵雍解释蒙卦为蒙昧闭塞，主回还往复，疑惑不前，多忧愁过失，乃是凶卦。

智慧未开，犹豫不决，缺乏果断。得此卦者，智慧犹如童蒙，须通过贤师良友之教，启其聪明则亨通。

蒙卦预示着时运蓄积德行，出而用世；财运则需要果断才能吉利；家宅吉祥，婚姻之始有利，身体驱去邪热可保平安。

传统解卦中提到，蒙卦形成山下有险的卦象，仍不停止前进，显示出启蒙和通达的特质。运势初时迷惑不知方向，需要忍耐待机而动，事业开始混乱无序，危机四伏，需要勇敢坚毅的行动来扭转局面。然而必须接受严格教育，培养这种奋发图强的精神。务必脚踏实地，最忌好高骛远，否则会陷入孤立无援的境地。经商务必小心谨慎，不得急功近利，尤其需树立高尚的商业道德，以良好的信誉提高竞争力。在决策时，有时会陷入迷惘困顿，加上胆小、不果断，往往误事。如能接受长辈的教诲，甚至严酷的考验，抛弃疑惧的心理，等待适当时机，必然一帆风顺。


In [41]:
import csv
import datetime
import os

def main():
    # 确保 data 目录存在
    if not os.path.exists('data'):
        os.makedirs('data')

    # 解析 data/raw_data.txt 得到 raw_content_data 列表
    raw_content_data = []
    with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
        content = file.read()
        data_samples = content.split('\n\n')
        for sample in data_samples:
            cleaned_sample = sample.strip()
            if cleaned_sample:
                raw_content_data.append(cleaned_sample)

    # 创建带有时间戳的CSV文件名
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"data/zhouyi_dataset_{timestamp}.csv"

    # 创建CSV文件并写入标题行
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['content', 'summary'])

        # 循环遍历 raw_content_data 数据样例
        for raw_content in raw_content_data:
            # 调用 gen_data 方法得到 ai_message_content
            ai_message_content = gen_data(raw_content)

            # 解析 ai_message_content 得到 content 和 summary
            content, summary = dataset_parser(ai_message_content)
            
            print("Content:", content)
            print("Summary:", summary)

            # 调用 generate_question_summary_pairs 得到20组 pairs
            pairs = generate_question_summary_pairs(content, summary)

            # 将 pairs 写入 csv 文件
            for pair in pairs:
                writer.writerow(pair)


In [43]:
# 执行主函数
main()

Content: 蒙卦
Summary: 蒙卦是周易中的一卦，象征着通泰。其卦象由下卦坎（水）和上卦艮（山）组成，寓意着山下有泉，泉水喷涌而出。卦辞告诫人们不应轻慢不敬地再三占筮，以免神灵不再告知。然而，卜问此卦仍然是吉利的。

蒙卦的核心哲学是：初时迷惑不知方向，需要等待时机，听取别人意见，才能获得通达和启蒙。

此卦在事业上表示，开始阶段可能会面临混乱无序和危机四伏，但凭借勇敢坚毅的行动可以扭转局面。然而必须接受严格教育，培养奋发图强的精神。务必脚踏实地，不可好高骛远，否则会陷入孤立无援的境地。在经商方面，务必小心谨慎，树立高尚的商业道德，以提高竞争力。在求名方面，必须接受良好的基础教育，陶冶情操。在婚恋方面，要注意考察对方品德，不可被金钱所诱惑。在决策上，如能接受长辈的教诲，抛弃疑惧心理，等待适当时机，必然一帆风顺。

总之，蒙卦是一卦通达启蒙之象，代表着在迷惘困顿之时及时接受教诲，等待时机，方能获得成功。
Content: {
  "content": "屯卦",
  "summary": "在周易中，屯卦代表着大吉大利，预示着吉利的占卜。然而，不利于外出活动，而利于建国封侯。这一卦象由上卦坎（水）和下卦震（雷）相叠而成，象征着云与雷的运动。君子观此卦象，应取法于云雷，用云的恩泽、雷的威严来治理国事。屯卦预示着困难和险境，需要刚毅果敢方可获得吉利。传统解卦中也强调，身处困境时，必须多加辛苦努力，并须有他人相助，方可通达。",
  "explanation": {
    "original": "屯。元，亨，利，贞。勿用，有攸往，利建侯。象曰：云，雷，屯；君子以经纶。",
    "白话文解释": "屯卦。大吉大利，吉利的占卜。不利于出门。有利于建国封侯。《象辞》说：屯的上卦为坎，坎为云，下卦为震，震为雷。云行于上，雷动于下，是屯卦的卦象。君子观此卦象，取法于云雷，用云的恩泽，雷的威严来治理国事。"
  },
  "解卦": {
    "断易天机": "屯卦坎上震下，为坎宫二世卦。屯说明有困难，又象征动而逢险境，需刚毅果敢方为吉。",
    "邵雍解": "万物始生，开始困难；先劳后逸，苦尽甘来。得此卦者，身处困境，宜守不宜进，须多加辛苦努力，排除困难，方可通达，有初难后解之象。",
    "傅佩荣解": "时运：宜守不宜进。财运：创业维艰。家宅：修缮

### 异常分析


训练第一个 epoch 时，Training Loss 比较奇怪：

```
Step	Training Loss
1	3.594100
2	4.049100
3	3.091200
4	3.381700
5	3.547800
6	2.610200
7	2.657900
8	3.163900
```

通过解析 GPT-3.5-Turbo-1106 生成结果发现问题

In [44]:
def gen_data(raw_content):
    """
    使用LangChain GPT-3.5调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-3.5模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

        返回格式要求：
        content:"{卦名}"
        summary:"{内容}"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

In [45]:
# 执行主函数
main()

Content: 蒙卦
Summary: 蒙卦是由艮卦（山）上卦坎卦（水）下相叠而成的卦象。它象征着通达，意在说明在某些情况下，即使我们似乎在向幼稚愚昧的人寻求帮助，实际上也是他们在向我们寻求帮助。第一次占卜可以获得神灵的指示，但如果轻慢不敬地反复占卜，神灵就不会再告诉我们。然而，总体上，这是一个吉利的卜问。

蒙卦的核心哲学是：在迷惑和无知之初，需要耐心等待机会，听取他人意见，运势才能通达。在事业和经商方面，需要勇敢和坚毅的行动来转变局面，同时接受严格的教育，培养奋发向上的精神。必须避免虚浮和冒进，才能取得成功。同时，对于求名和婚恋，也要注重基础教育和内在品德的培养，以及真诚和宽容的态度。
Content: 屯卦
Summary: 在周易中，屯卦代表着大吉大利，预示着吉利的占卜结果。然而，不利于出门，却有利于建国封侯。卦象上卦为坎，坎为云，下卦为震，震为雷，云行于上，雷动于下，象征着动而逢险境，需刚毅果敢方为吉。屯卦在占卜中指示身处困境，宜守不宜进，须多加辛苦努力排除困难，方可通达，有初难后解之象。在事业上，要知难而进，小心翼翼，勇往直前，灵活机动，可望获得大的成功，但也需有他人相助。在经商方面，创业初期步履艰难，多有挫折，需要坚定信念，积极进取，行动果断，如无法摆脱困境，则应退守保全，等待机会再展宏图。而在婚恋方面，忠贞纯洁，大胆追求将能够获得成功，婚姻也将美满。总之，屯卦代表了初始困难，但若具有坚忍不拔的毅力和锲而不舍的奋斗精神，前途不可估量，但往往需要他人的帮助才能摆脱困境。
Content: 需卦
Summary: 在周易中，需卦是由上卦坎（水）和下卦乾（天）组成的异卦。这一卦象代表抓到俘虏，预示着大吉大利，适宜进行吉利的卜问以及涉水渡河。《象辞》中描述云浮聚于天上，待时降雨，象征着等待时机的需卦。君子观此卦象，可以宴饮安乐，待时而动。

需卦暗示着踌躇和期待，虽然刚强，但需要等待时机，涉大川则会顺利。传统解卦中指出，智者必须待时而行，不可急进，急进反见凶险。需要审时度势，耐心等待，切勿冒险，欲速不达。在事业、经商、求名、婚恋和决策等方面，均需要谨慎等待时机，耐心前行。
Content: 讼卦
Summary: 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，乾象征刚健，坎象征险陷。这两者相背而行，预示着将会有争讼之事。占卦得此卦，